In [5]:
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch
import numpy as np
import pickle
from tqdm import tqdm

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load meta
with open(r"D:\food_recommender\embeddings\meta_clean_original_paths.pkl", "rb") as f:
    meta = pickle.load(f)

# Embed the NEXT 1000 images AFTER index 8000 → indices 8000 to 9000
start_idx = 8000
end_idx = start_idx + 1000  # 9000

image_paths = meta["image_path"][start_idx:end_idx]

image_embeds = []

for img_path in tqdm(image_paths, desc=f"Embedding images {start_idx}–{end_idx}"):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        emb = model.get_image_features(**inputs)

    emb = emb / emb.norm(dim=-1, keepdim=True)
    image_embeds.append(emb.cpu().numpy())

image_embeds = np.vstack(image_embeds).astype("float32")

np.save(r"D:\food_recommender\embeddings\image_embeds_8000_to_9000.npy", image_embeds)

print("DONE:", image_embeds.shape)



Embedding images 8000–9000: 100%|██████████| 1000/1000 [1:08:59<00:00,  4.14s/it]

DONE: (1000, 768)


In [6]:
import numpy as np
import faiss
import pickle

# ============================
#  Load embeddings + meta
# ============================
EMB_PATH = r"D:\food_recommender\embeddings\image_embeds_8000_to_9000.npy"

# ============================
#  Create FAISS index
# ============================
embeds = np.load(EMB_PATH).astype("float32")
d = embeds.shape[1]   # embedding dimension
index = faiss.IndexFlatL2(d)  # L2 cosine-normalized = works well

index.add(embeds)  # add all embeddings
print("FAISS index size:", index.ntotal)

# ============================
#  Save index
# ============================
faiss.write_index(index, r"D:/food_recommender/embeddings/faiss_image_index_cleaned_8000_to_9000.bin")
print("Saved FAISS index → faiss_image.index")


FAISS index size: 1000
Saved FAISS index → faiss_image.index


In [10]:
import faiss
import numpy as np
import os

# ============================
#  CONFIG
# ============================
INDEX_FILES = [
    r"D:\food_recommender\embeddings\faiss_image_index_cleaned.bin",
    r"D:\food_recommender\embeddings\faiss_image_index_cleaned_500_to_2000.bin",
    r"D:\food_recommender\embeddings\faiss_image_index_cleaned_2000_to_4000.bin",
    r"D:\food_recommender\embeddings\faiss_image_index_cleaned_4000_to_5000.bin",
    r"D:\food_recommender\embeddings\faiss_image_index_cleaned_5000_to_7999.bin",
    r"D:\food_recommender\embeddings\faiss_image_index_cleaned_8000_to_9000.bin",
    r"D:\food_recommender\embeddings\faiss_image_index_cleaned_9000_to_end.bin"
]

OUT_INDEX = r"D:\food_recommender\embeddings\faiss_all_images.bin"

# ============================
#  LOAD BASE INDEX
# ============================
base_index = faiss.read_index(INDEX_FILES[0])
d = base_index.d

print(f"Loaded base index: {INDEX_FILES[0]} ({base_index.ntotal} vectors)")

# ============================
#  APPEND OTHERS
# ============================
for path in INDEX_FILES[1:]:
    assert os.path.exists(path), f"❌ Missing file: {path}"

    idx = faiss.read_index(path)

    # Sanity checks
    assert idx.d == d, "❌ Dimension mismatch"
    assert type(idx) == type(base_index), "❌ Index type mismatch"

    # 🔥 Correct FAISS merge
    base_index.add(idx.reconstruct_n(0, idx.ntotal))

    print(f"Appended {idx.ntotal} vectors from {path}")

# ============================
#  SAVE
# ============================
faiss.write_index(base_index, OUT_INDEX)

print("=================================")
print("✅ Final index size:", base_index.ntotal)
print("💾 Saved combined index →", OUT_INDEX)


Loaded base index: D:\food_recommender\embeddings\faiss_image_index_cleaned.bin (500 vectors)
Appended 1500 vectors from D:\food_recommender\embeddings\faiss_image_index_cleaned_500_to_2000.bin
Appended 2000 vectors from D:\food_recommender\embeddings\faiss_image_index_cleaned_2000_to_4000.bin
Appended 1000 vectors from D:\food_recommender\embeddings\faiss_image_index_cleaned_4000_to_5000.bin
Appended 3000 vectors from D:\food_recommender\embeddings\faiss_image_index_cleaned_5000_to_7999.bin
Appended 1000 vectors from D:\food_recommender\embeddings\faiss_image_index_cleaned_8000_to_9000.bin
Appended 4471 vectors from D:\food_recommender\embeddings\faiss_image_index_cleaned_9000_to_end.bin
✅ Final index size: 13471
💾 Saved combined index → D:\food_recommender\embeddings\faiss_all_images.bin


In [5]:
import faiss
import pickle
import numpy as np

# ============================
# CONFIG
# ============================
FAISS_INDEX_PATH = r"D:\food_recommender\embeddings\faiss_all_images.index"
META_PATH = r"D:\food_recommender\embeddings\meta_clean_original_paths.pkl"

# ============================
# LOAD
# ============================
index = faiss.read_index(FAISS_INDEX_PATH)

with open(META_PATH, "rb") as f:
    meta = pickle.load(f)

print("FAISS vectors:", index.ntotal)
print("Meta rows:", len(meta["image_path"]))


FAISS vectors: 13471
Meta rows: 13471


In [6]:
assert index.ntotal == len(meta["image_path"]), \
    "❌ ALIGNMENT ERROR: FAISS index and meta size mismatch"

print("✅ Size check passed")


✅ Size check passed


In [9]:
def verify_alignment(index, num_checks=50):
    dim = index.d

    for i in np.random.choice(index.ntotal, size=num_checks, replace=False):
        i = int(i)
        vec = index.reconstruct(i).reshape(1, dim)
        _, I = index.search(vec, k=1)

        if I[0][0] != i:
            raise RuntimeError(f"❌ Vector mismatch at index {i}")

    print(f"✅ Passed {num_checks} self-retrieval checks")

verify_alignment(index)


✅ Passed 50 self-retrieval checks
